# 📘 Next-Word Prediction using MLP (War and Peace)

In [ ]:
# ✅ Install Dependencies (Colab only)
!pip install -q tensorflow


In [ ]:
# ✅ Imports
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests
import pickle


In [ ]:
# ✅ Download and Clean Text (War and Peace)
def download_text():
    url = "https://www.gutenberg.org/files/2600/2600-0.txt"
    response = requests.get(url)
    text = response.text.lower()
    return re.sub(r'[^a-zA-Z0-9 \.]', '', text)

text = download_text()
sentences = text.split('.')
sentences = [s.strip() for s in sentences if len(s.strip().split()) > 5]
print(f"Total sentences: {len(sentences)}")


Total sentences: 23843


In [ ]:
# ✅ Tokenize into Words
context_len = 5
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocab size: {vocab_size}")


Vocab size: 46190


In [ ]:
# ✅ Create (X, y) Word Sequences
sequences = []
for sentence in sentences:
    words = sentence.split()
    for i in range(context_len, len(words)):
        context = words[i - context_len:i]
        target = words[i]
        sequences.append((context, target))

X = []
y = []
for context, target in sequences:
    ctx_seq = tokenizer.texts_to_sequences([' '.join(context)])[0]
    target_seq = tokenizer.texts_to_sequences([target])[0][0]
    X.append(ctx_seq)
    y.append(target_seq)

X = np.array(X)
y = np.array(y)
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (395873, 5)
Shape of y: (395873,)


In [ ]:
# ✅ Define and Train MLP Model
embedding_dim = 64
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_shape=(context_len,), name="embedding"),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(vocab_size, activation='softmax')
])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 5, 64)          │     2,956,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 46190)          │    47,344,750 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,629,614 (193.14 MB)

 Trainable params: 50,629,614 (193.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ✅ Train the model (use GPU on Colab)
model.fit(X, y, epochs=50, batch_size=512)


Epoch 1/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 49s 61ms/step - accuracy: 0.0758 - loss: 7.7071
Epoch 2/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 80s 60ms/step - accuracy: 0.1333 - loss: 6.1897
Epoch 3/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.1651 - loss: 5.5416
Epoch 4/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.2031 - loss: 4.8395
Epoch 5/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 60ms/step - accuracy: 0.2556 - loss: 3.9580
Epoch 6/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.4003 - loss: 3.0393
Epoch 7/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.5000 - loss: 2.4400
Epoch 8/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.5807 - loss: 2.0029
Epoch 9/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.6432 - loss: 1.6815
Epoch 10/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.6941 - loss: 1.4195
Epoch 11/50
774/774 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.7340 - loss: 1.2157
Epoch 12/50
774/774 ━━━━━━━━━━

In [ ]:
# ✅ Save model and tokenizer for use in Streamlit
model.save("mlp_model.h5")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
print("Saved model and tokenizer.")


Saved model and tokenizer.


In [ ]:
# ✅ Verify GPU is being used
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1
